# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [4]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities.rename(columns={"Population (2016 est.)[8]": "Population"}, inplace=True) # Rename the column
cities['NFL'] = cities['NFL'].str.replace(r"\[.*\]", "") # Get rid of square brackets in team names
cities['MLB'] = cities['MLB'].str.replace(r"\[.*\]", "")
cities['NBA'] = cities['NBA'].str.replace(r"\[.*\]", "")
cities['NHL'] = cities['NHL'].str.replace(r"\[.*\]", "")

league = 'NHL'

"""
Prepares a dataframe with the name of the teams for each metropolitan area
"""


def prepare_teams_df():
    # divide team names into separate teams (because they were given as one string)
    teams = cities[league].str.extract('([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)')
    # add the name of the area column
    teams['Metropolitan area'] = cities['Metropolitan area']
    # change df from wide to long
    teams = pd.melt(teams, id_vars=['Metropolitan area'])
    # drop a useless column
    teams = teams.drop(columns=['variable'])
    # change empty cells into np.nan
    teams = teams.replace("", np.nan).replace("—", np.nan).dropna()
    # reset index to default (because before pd.melt it was a different dataframe)
    teams = teams.reset_index()
    # drop the index column from resetting the index
    teams = teams.drop(columns=['index'])
    # rename the last column
    teams = teams.rename(columns={"value": "team"})

    # merge two dataframes to get the population column
    teams = pd.merge(teams, cities, how="left", on="Metropolitan area")
    # get only necessary columns (team, Population, NFL)
    teams = teams[['Metropolitan area', 'team', 'Population', 'NHL']]
    teams = teams.astype({'Metropolitan area': str, 'team': str, 'Population': int})
    
    return teams

def nhl_correlation():    
    teams = prepare_teams_df()

    nhl_df=pd.read_csv("assets/nhl.csv")
    
    # take only year 2018
    nhl_df = nhl_df[nhl_df['year'] == 2018]
    # get rid of * at the end of team names
    nhl_df['team'] = nhl_df['team'].str.replace(r"\*", "")
    # get only team name, number of wins, number of losses
    nhl_df = nhl_df[['team', 'W', 'L']]

    # keep only actual teams, drop region names (Central Division, etc.)
    nhl_df = nhl_df[nhl_df['team'] != nhl_df['W']]

    # set index to be Metropolitan Area
    teams.set_index('Metropolitan area', inplace=True)
    # save original indecies
    or_indecies = teams.index
    # rename areas to match team names
    teams = teams.rename(index={"New York City": "New York", 
                                "Washington, D.C.": "Washington", 
                                "San Francisco Bay Area": "San Jose",
                                "Tampa Bay Area": "Tampa Bay",
                                "Denver": "Colorado",
                                "Minneapolis–Saint Paul": "Minnesota",
                                "Dallas–Fort Worth": "Dallas",
                                "Miami–Fort Lauderdale": "Florida",
                                "Phoenix": "Arizona",
                                "Las Vegas": "Vegas",
                                "Raleigh": "Carolina"})

    teams = teams.reset_index()
    # add area names to the team names
    teams['team'] = teams['Metropolitan area'] + " " +  teams['team']

    teams.iloc[30, 1] = 'New Jersey Devils'
    teams.iloc[29, 1] = 'Anaheim Ducks'

    nhl_df = nhl_df.astype({'team': str, 'W': int, 'L': int})
    nhl_df['W/L%'] = nhl_df['W'] / (nhl_df['W'] + nhl_df['L'])

    # merge teams with their wins and losses
    merged_df = pd.merge(teams, nhl_df, how='outer', on='team')
    merged_df['Metropolitan area'] = or_indecies

    merged_df = merged_df.groupby('Metropolitan area').agg({'W/L%': np.nanmean, 'Population': np.nanmean})
    
    population_by_region = merged_df['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = merged_df['W/L%'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
        
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

cities


<ipython-input-4-4068cb7f3194>:9: FutureWarning: The default value of regex will change from True to False in a future version.
  cities['NFL'] = cities['NFL'].str.replace(r"\[.*\]", "") # Get rid of square brackets in team names
<ipython-input-4-4068cb7f3194>:10: FutureWarning: The default value of regex will change from True to False in a future version.
  cities['MLB'] = cities['MLB'].str.replace(r"\[.*\]", "")
<ipython-input-4-4068cb7f3194>:11: FutureWarning: The default value of regex will change from True to False in a future version.
  cities['NBA'] = cities['NBA'].str.replace(r"\[.*\]", "")
<ipython-input-4-4068cb7f3194>:12: FutureWarning: The default value of regex will change from True to False in a future version.
  cities['NHL'] = cities['NHL'].str.replace(r"\[.*\]", "")


,Metropolitan area,Population,NFL,MLB,NBA,NHL
0,New York City,20153634,GiantsJets,YankeesMets,KnicksNets,RangersIslandersDevils
1,Los Angeles,13310447,RamsChargers,DodgersAngels,LakersClippers,KingsDucks
2,Chicago,9512999,Bears,CubsWhite Sox,Bulls,Blackhawks
3,San Francisco Bay Area,6657982,49ers,GiantsAthletics,Warriors,Sharks
4,Dallas–Fort Worth,7233323,Cowboys,Rangers,Mavericks,Stars
5,"Washington, D.C.",6131977,Washington Football Team,Nationals,Wizards,Capitals
6,Philadelphia,6070500,Eagles,Phillies,76ers,Flyers
7,Miami–Fort Lauderdale,6066387,Dolphins,Marlins,Heat,Panthers
8,Boston,4794447,Patriots,Red Sox,Celtics,Bruins
9,Minneapolis–Saint Paul,3551036,Vikings,Twins,Timberwolves,Wild


## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities.rename(columns={"Population (2016 est.)[8]": "Population"}, inplace=True) # Rename the column
cities['NFL'] = cities['NFL'].str.replace(r"\[.*\]", "") # Get rid of square brackets in team names
cities['MLB'] = cities['MLB'].str.replace(r"\[.*\]", "")
cities['NBA'] = cities['NBA'].str.replace(r"\[.*\]", "")
cities['NHL'] = cities['NHL'].str.replace(r"\[.*\]", "")

league = 'NBA'

def prepare_teams_df():
    # divide team names into separate teams (because they were given as one string)
    teams = cities[league].str.extract('([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)')
    # add the name of the area column
    teams['Metropolitan area'] = cities['Metropolitan area']
    # change df from wide to long
    teams = pd.melt(teams, id_vars=['Metropolitan area'])
    # drop a useless column
    teams = teams.drop(columns=['variable'])
    # change empty cells into np.nan
    teams = teams.replace("", np.nan).replace("—", np.nan).dropna()
    # reset index to default (because before pd.melt it was a different dataframe)
    teams = teams.reset_index()
    # drop the index column from resetting the index
    teams = teams.drop(columns=['index'])
    # rename the last column
    teams = teams.rename(columns={"value": "team"})

    # merge two dataframes to get the population column
    teams = pd.merge(teams, cities, how="left", on="Metropolitan area")
    # get only necessary columns (team, Population, NFL)
    teams = teams[['Metropolitan area', 'team', 'Population', 'NBA']]
    teams = teams.astype({'Metropolitan area': str, 'team': str, 'Population': int})
    
    return teams

def nba_correlation():
    teams = prepare_teams_df()

    nba_df=pd.read_csv("assets/nba.csv")
    
    # take only year 2018
    nba_df = nba_df[nba_df['year'] == 2018]
    # get rid of * at the end of team names
    nba_df['team'] = nba_df['team'].str.replace(r"\*", "")
    # get rid of (numbers) at the end of the team names
    nba_df['team'] = nba_df['team'].str.replace(r"\(.*\)", "")
    # get only team name, number of wins, number of losses
    nba_df = nba_df[['team', 'W', 'L']]

   
        
    # population_by_region = merged_df['Population'] # pass in metropolitan area population from cities
    # win_loss_by_region = merged_df['W/L%'] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

#     assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
#     assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return nba_df
    
    # return stats.pearsonr(population_by_region, win_loss_by_region)

nba_correlation()

ValueError: No tables found

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def mlb_correlation(): 
    # YOUR CODE HERE
    raise NotImplementedError()
    
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nfl_correlation(): 
    # YOUR CODE HERE
    raise NotImplementedError()
    
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    # YOUR CODE HERE
    raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values